# Comprehensive Quantitative Research: Advanced Stock Data Cleaning & Smoothing
This report provides an exhaustive analysis of 20+ algorithms used to handle missingness, outliers, and noise in high-dimensional financial time series data.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Add project root to path
sys.path.append(os.path.abspath('..'))

## 1. Initial Data Quality Assessment

### Intuition
Raw financial data is rarely ready for modeling. Gaps and integrity violations can lead to 'Garbage In, Garbage Out'. We quantify the dataset's 'badness' using physical and statistical constraints.

### Mathematical Definitions
1. **Missing Pct**: Measures the 'Swiss Cheese' effect in data.
   $$\text{Missing Pct} = \frac{\sum_{t=1}^T \mathbb{1}(x_t = \text{NaN})}{T} \times 100$$
   *Variable*: $T$ is total possible time steps.

2. **OHLC Physical Integrity**: Financial prices must follow a strict hierarchy.
   - **Constraint**: $H_t \ge \{O_t, C_t, L_t\}$ and $L_t \le \{O_t, C_t, H_t\}$.
   - **Intuition**: A price cannot be higher than the day's maximum or lower than its minimum. Violations indicate corrupted data feeds.

3. **Zero-Print Check**: Count of records where $P_t = 0$.
   - **Intuition**: Liquid assets do not trade at zero. These are usually query failure artifacts.

In [ ]:
quality_report = pd.read_csv('../research_quality_report.csv')
pd.set_option('display.max_rows', 100)
quality_report

## 2. Missing Value Imputation Survey

### Intuition
Modern models require continuous vectors. Imputation fills gaps by estimating the most likely values based on surrounding data or global distribution.

### Algorithm & Mathematical Breakdown

1. **Mean Imputation**: Replaces NaNs with the global average $\bar{x}$.
   $$\hat{x}_t = \frac{1}{N} \sum x_i$$
   - **Variable**: $N$ is count of non-NaN values.

2. **Linear Interpolation**: Assumes the asset moves at a constant velocity between two known points.
   $$x_t = x_a + (x_b - x_a) \frac{t-a}{b-a}$$
   - **Variables**: $a, b$ are indices of known points; $t$ is missing index.

3. **Forward Fill**: Assumes the market is static until the next print.
   $$x_t = x_{t-1}$$

4. **Maximum Likelihood (MLE)**: Finds $\mu, \sigma$ that maximize the probability of the seen data, then uses $\mu$ to fill NaNs.
   $$\mathcal{L}(\mu, \sigma) = \prod_{i=1}^n \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{(x_i - \mu)^2}{2\sigma^2}}$$

5. **Expectation-Maximization (EM)**: An iterative two-step process.
   - **E-Step**: Calculate $E[x_{miss} | x_{obs}, \theta]$.
   - **M-Step**: Maximize $\theta = (\mu, \sigma)$ using the completed data.

6. **GAN (Generative Adversarial Network)**: A Generator $G$ learns to 'draw' stock movements to fool a Discriminator $D$.
   - **Math**: $\min_G \max_D \mathbb{E}[\log D(x)] + \mathbb{E}[\log(1 - D(G(z)))]$

In [ ]:
results = pd.read_csv('../research_algorithm_comparison.csv', index_col=0)
imputation_results = results.loc[['Mean', 'Interpolation', 'Forward_Fill', 'GAN', 'MLE', 'EM']]
imputation_results.plot(kind='bar', title='Imputation Masked MSE')

## 3. Anomaly Detection Comparison

### Intuition
We separate signal from artifacts. Anomalies are points that deviate from expected statistical behavior.

### Algorithm & Variable Dictionary

1. **3-Sigma**: Identifies values where $|x_t - \mu| > 3\sigma$.
   - **Intuition**: In a Normal distribution, this covers 99.7% of data. Anything outside is an extreme outlier.

2. **IQR (Interquartile Range)**: Uses quartiles to avoid being biased by the outliers.
   - **Math**: Outlier if $x_t < Q_1 - 1.5 \cdot IQR$ or $x_t > Q_3 + 1.5 \cdot IQR$.

3. **Isolation Forest**: Built on the idea that outliers are few and different.
   - **Mechanism**: Randomly splits the data. Outliers end up in shorter branches (easier to isolate).
   - **Variable**: Path length $h(x)$.

4. **LOF (Local Outlier Factor)**: Measures the density of a point compared to its neighbors.
   - **Math**: $LOF(k) = \frac{\text{avg neighbor density}}{\text{local density}}$. Score $>1$ is sparse.

5. **DBSCAN**: Density-based clustering.
   - **Variables**: $\epsilon$ (radius), $MinPts$ (min points in radius).

6. **Window Anomaly**: Adapts to changing market conditions.
   - **Math**: $z_t = \frac{x_t - \text{roll\_mean}_k}{\text{roll\_std}_k}$.

7. **Abnormal Sequence**: Detects volatility clusters.
   - **Math**: $\text{Flag if } \text{Var}_{win} > \tau \cdot \text{Var}_{global}$.

In [ ]:
outlier_df = pd.read_csv('../research_outlier_report.csv')
outlier_df.set_index('Asset').plot(kind='box', title='Anomaly Detector Sensitivity')

## 4. Advanced Smoothing & Statistical Survey

### Intuition
Smoothing recovers the latent 'signal' $s_t$ from noisy observations $x_t = s_t + \epsilon_t$.

### Mathematical Formalism & Variable Dictionary

1. **SMA (Simple Moving Average)**:
   - **Math**: $y_t = \frac{1}{k} \sum_{i=0}^{k-1} x_{t-i}$
   - **Variables**: $k$ is window size.

2. **EMA (Exponential Moving Average)**:
   - **Math**: $y_t = \alpha x_t + (1-\alpha) y_{t-1}$
   - **Intuition**: Gives more weight to recent data. $\alpha = \frac{2}{k+1}$.

3. **Kalman Filter**:
   - **Variables**: $x$ (state), $z$ (observation), $P$ (error covariance), $Q$ (process noise), $R$ (sensor noise).
   - **Predict Step**: $\hat{x}_{k}^- = \hat{x}_{k-1}$.
   - **Update Step**: $K_k = P_{k}^- (P_k^- + R)^{-1}$, $\hat{x}_k = \hat{x}_k^- + K_k (z_k - \hat{x}_k^-)$.

4. **Wiener Filter**:
   - **Intuition**: Optimal stationary filter. Acts on the Power Spectral Density $S_{xx}(f)$.
   - **Math**: $H(f) = \frac{S_{xx}(f)}{S_{xx}(f) + S_{nn}(f)}$.

5. **Lattice Filter (Levinson-Durbin)**:
   - **Intuition**: Models signal reflections. $k_m$ are reflection coefficients.
   - **Math**: $f_m(n) = f_{m-1}(n) + k_m b_{m-1}(n-1)$.

6. **HMM Smoothing**:
   - **Math**: Returns $\mathbb{E}[x_t | S_t]$ where $S_t$ is the most likely hidden regime path found via the Viterbi algorithm.

7. **LMS Adaptive Filter**:
   - **Math**: $w_{n+1} = w_n + 2 \mu e_n x_n$. Updates weights via gradient descent.

8. **Bayesian Smoothing**:
   - **Math**: $\mu_{post} = \frac{\mu_{prior}/\sigma_{prior}^2 + x/\sigma_{noise}^2}{1/\sigma_{prior}^2 + 1/\sigma_{noise}^2}$.

9. **Markov Model**:
   - **Math**: $\hat{x}_{t+1} = \sum P(S_{t+1} | S_t) \cdot \text{Value}(S_{t+1})$.

10. **SMURF**:
    - **Intuition**: Robust trend extraction via rolling median of residuals.

11. **PCA (Relationship Dependent)**:
    - **Math**: $X = W Z + E$. Reconstruction uses common factors $W Z$ to filter idiosyncratic noise $E$.

12. **AR / ARMA**:
    - **Math**: $x_t = c + \sum \phi_i x_{t-i} + \sum \theta_j \epsilon_{t-j}$.

In [ ]:
smooth_results = pd.read_csv('../research_smoothing_metrics.csv', index_col=0)
fig, axes = plt.subplots(3, 1, figsize=(12, 18))
smooth_results['RMSE'].plot(kind='bar', ax=axes[0], title='Smoothing RMSE (Lower is Better)')
smooth_results['LAG'].plot(kind='bar', ax=axes[1], title='Lag (Lower is Better)')
smooth_results['RPR'].plot(kind='bar', ax=axes[2], title='Smoothness (RPR - Lower is Better)')
plt.tight_layout()

## 5. Deep Learning Models

### LSTM Autoencoder
- **Intuition**: Forces the sequence through a temporal 'bottleneck'. Signal structures are preserved while stochastic noise is removed.
- **Math**: $\mathcal{L} = \| X - \text{Decoder}(\text{Encoder}(X)) \|^2$

## 6. Corporate Action Detection

### Intuition
Structural breaks (splits) create price gaps. Detecting them prevents cleaners from incorrectly 'fixing' real market structure.
- **Math**: Overnight Log-Return $R_t = \ln(P_t/P_{t-1})$. If $R_t < \tau$, flag as potential split.

In [ ]:
pd.read_csv('../research_corporate_report.csv')['Potential_Splits'].plot(kind='hist', bins=20, title='Potential Splits Frequency')

## Conclusion
Advanced statistical models (EM, HMM) and adaptive filters (Wiener) provide the most robust noise reduction for these assets.